In [1]:
from tqdm import tqdm
import numpy as np
import os
import pandas as pd
import time
import warnings
import wfdb
# 2s

In [2]:
def find_pdf_files(folder_path, include_subfolders=True):

    pdf_files = []
    if include_subfolders:
        # Walk through the directory and its subdirectories
        for root, dirs, files in os.walk(folder_path):
            for file in files:
                if file.endswith('n.hea'):
                    1+1
                elif '_' in str(file):
                    1+1
                elif '_layout' in str(file):
                    1+1
                elif file.endswith('.hea'):
                    pdf_files.append(os.path.join(root, file))
    else:
        # Only check files in the specified directory (not subdirectories)
        for file in os.listdir(folder_path):
            if file.endswith('n.hea'):
                1+1
            elif '_' in str(file):
                1+1
            elif '_layout' in str(file):
                1+1
            elif file.endswith('.hea'):
                pdf_files.append(os.path.join(folder_path, file))
    return pdf_files

In [3]:
hea_files = find_pdf_files('M:\\', include_subfolders=True)
print(len(hea_files))
hea_files = set(hea_files)
hea_files = list(hea_files)
hea_files.sort()

apth = []
for ii in hea_files:
    ii = str(ii)
    if '_0' in ii:
        # print(ii)
        1+1
    else:
        apth.append(ii)
print(len(apth))
# 36s

22317
22317


In [4]:
findDf = pd.DataFrame()

findDf['xIndex'] = range(len(apth))
findDf['subject_id'] = ''
findDf['path'] = apth

findDf['subject_id'] = findDf['path'].str.split('\\', expand=True)[3]
findDf['subject_id'] = findDf['subject_id'].str.split('-', expand=True)[0]
findDf['subject_id'] = findDf['subject_id'].str.replace('p', '')
findDf['subject_id'] = findDf['subject_id'].astype('string')
findDf['subject_id'] = findDf['subject_id'].str.lower().str.strip()
findDf['subject_id_int64'] = findDf['subject_id'].astype(np.int64)

findDf['loadPath'] = apth
findDf['loadPath'] = findDf['loadPath'].str.replace('.hea', '')

findDf['ecgDate'] = ''
for ii in range(len(findDf)):
    try:
        st = str(findDf.iloc[ii]['path']).find('-')+1
        en = str(findDf.iloc[ii]['path']).find('.hea')
        strrr = str(findDf.iloc[ii]['path'])
        findDf.loc[ii, 'ecgDate'] = strrr[st:en]
    except:
        print('error')
        print(str(ii))
        print(findDf.iloc[ii]['path'])

findDf['ecgDate'] = pd.to_datetime(findDf['ecgDate'], format='%Y-%m-%d-%H-%M')

findDf['ecgDatetime'] = ''
findDf['sig_name'] = ''
# findDf['sig_length'] = ''
findDf['fs'] = ''
for ii in tqdm(range(len(findDf))):
    try:
        tmp = wfdb.rdrecord(findDf.iloc[ii]['loadPath'],
                            sampfrom=0, sampto=1,
                            smooth_frames=False)
        spp = str(tmp.base_datetime).split('.')
        findDf.loc[ii, 'ecgDatetime'] = spp[0]
        findDf.loc[ii, 'sig_name'] = str(tmp.sig_name)
        # findDf.loc[ii, 'sig_length'] = tmp.sig_len
        findDf.loc[ii, 'fs'] = tmp.fs
    except:
        print('wfdb read error')
        print(findDf.iloc[ii]['loadPath'])
        print()


findDf['ecgDatetime'] = pd.to_datetime(
    findDf['ecgDatetime'], format='%Y-%m-%d %H:%M:%S')
# 34m5s

 32%|███▏      | 7194/22317 [03:04<07:46, 32.39it/s]

wfdb read error
M:\p02\p029553\p029553-2197-05-12-21-19

wfdb read error
M:\p02\p029553\p029553-2197-05-13-00-15

wfdb read error
M:\p02\p029553\p029553-2197-05-13-02-28

wfdb read error
M:\p02\p029553\p029553-2197-05-13-04-38



 94%|█████████▎| 20903/22317 [09:49<00:34, 40.86it/s]

wfdb read error
M:\p09\p094029\p094029-2109-02-23-18-52



 97%|█████████▋| 21593/22317 [10:08<00:19, 37.03it/s]

wfdb read error
M:\p09\p096928\p096928-2139-01-02-10-04



100%|██████████| 22317/22317 [10:28<00:00, 35.53it/s]


In [5]:
findDf.dtypes

xIndex                       int64
subject_id          string[python]
path                        object
subject_id_int64             int64
loadPath                    object
ecgDate             datetime64[ns]
ecgDatetime         datetime64[ns]
sig_name                    object
fs                          object
dtype: object

In [6]:
print(findDf.shape)
print(len(pd.unique(findDf['subject_id'])))

(22317, 9)
10099


In [7]:
findDf['subject_id'].value_counts()

subject_id
048872    58
029035    24
019059    24
049555    23
095201    23
          ..
099503     1
099464     1
099417     1
000703     1
000682     1
Name: count, Length: 10099, dtype: Int64

In [8]:
findDf['fs'].value_counts()

fs
125    22311
           6
Name: count, dtype: int64

In [9]:
findDf[:10]

,xIndex,subject_id,path,subject_id_int64,loadPath,ecgDate,ecgDatetime,sig_name,fs
0,0,000020,M:\p00\p000020\p000020-2183-04-28-17-47.hea,20,M:\p00\p000020\p000020-2183-04-28-17-47,2183-04-28 17:47:00,2183-04-28 17:47:59,"['II', 'AVF', 'ABP', 'PAP']",125
1,1,000030,M:\p00\p000030\p000030-2172-10-16-12-22.hea,30,M:\p00\p000030\p000030-2172-10-16-12-22,2172-10-16 12:22:00,2172-10-16 12:22:07,"['II', 'V']",125
2,2,000033,M:\p00\p000033\p000033-2116-12-24-12-35.hea,33,M:\p00\p000033\p000033-2116-12-24-12-35,2116-12-24 12:35:00,2116-12-24 12:35:06,"['II', 'MCL1']",125
3,3,000033,M:\p00\p000033\p000033-2116-12-25-13-11.hea,33,M:\p00\p000033\p000033-2116-12-25-13-11,2116-12-25 13:11:00,2116-12-25 13:11:06,"['II', 'MCL1']",125
4,4,000052,M:\p00\p000052\p000052-2191-01-10-02-21.hea,52,M:\p00\p000052\p000052-2191-01-10-02-21,2191-01-10 02:21:00,2191-01-10 02:21:23,"['II', 'AVR', 'V', 'RESP', 'PLETH']",125
5,5,000052,M:\p00\p000052\p000052-2191-01-10-12-15.hea,52,M:\p00\p000052\p000052-2191-01-10-12-15,2191-01-10 12:15:00,2191-01-10 12:15:25,"['II', 'III', 'AVR', 'V', 'RESP', 'PLETH']",125
6,6,000079,M:\p00\p000079\p000079-2175-09-26-01-25.hea,79,M:\p00\p000079\p000079-2175-09-26-01-25,2175-09-26 01:25:00,2175-09-26 01:25:00,"['II', 'ABP', 'PAP']",125
7,7,000079,M:\p00\p000079\p000079-2175-09-26-12-28.hea,79,M:\p00\p000079\p000079-2175-09-26-12-28,2175-09-26 12:28:00,2175-09-26 12:28:00,"['II', 'V', 'MCL1', 'ABP', 'PAP']",125
8,8,000085,M:\p00\p000085\p000085-2167-07-25-21-11.hea,85,M:\p00\p000085\p000085-2167-07-25-21-11,2167-07-25 21:11:00,2167-07-25 21:11:31,"['II', 'V', 'III', 'I']",125
9,9,000107,M:\p00\p000107\p000107-2121-11-30-20-03.hea,107,M:\p00\p000107\p000107-2121-11-30-20-03,2121-11-30 20:03:00,2121-11-30 20:03:22,"['II', 'AVR', 'V', 'RESP', 'PLETH']",125


In [10]:
findDf.iloc[88]

xIndex                                                       88
subject_id                                               000416
path                M:\p00\p000416\p000416-2132-07-23-15-14.hea
subject_id_int64                                            416
loadPath                M:\p00\p000416\p000416-2132-07-23-15-14
ecgDate                                     2132-07-23 15:14:00
ecgDatetime                                 2132-07-23 15:14:06
sig_name                                     ['II', 'V', 'ABP']
fs                                                          125
Name: 88, dtype: object

In [11]:
def ioTimes(inDf):
    inDf = inDf.sort_values(by=['intime']).reset_index(drop=True)
    inDf['numberCountIcu'] = range(len(inDf))
    inDf['numberCountIcu'] = inDf['numberCountIcu'] + 1
    inDf['totalCountIcu'] = len(inDf)
    return inDf

In [12]:
icustays = pd.read_csv('N:\\ICUSTAYS.csv')
tmp = pd.DataFrame()
tmp['columns'] = icustays.columns
tmp['columns'] = tmp['columns'].str.lower().str.strip()
icustays.columns = tmp['columns'].to_list()

icustays['subject_id'] = icustays['subject_id'].astype('string')
icustays['subject_id'] = icustays['subject_id'].str.lower().str.strip()
icustays = icustays.dropna(subset=['subject_id', 'hadm_id'])
icustays['subject_id_int64'] = icustays['subject_id'].astype(np.int64)
icustays = icustays[icustays['subject_id_int64'].isin(
    findDf['subject_id_int64'])]

icustays['intime'] = pd.to_datetime(
    icustays['intime'], format='%Y-%m-%d %H:%M:%S')
icustays['outtime'] = pd.to_datetime(
    icustays['outtime'], format='%Y-%m-%d %H:%M:%S')


icustays = icustays.sort_values(
    by=['subject_id', 'intime', 'outtime', 'hadm_id', 'icustay_id']).reset_index(drop=True)

icustays00 = icustays.groupby(['subject_id', 'hadm_id']).apply(
    ioTimes).reset_index(drop=True)

icustays = icustays00.sort_values(
    by=['subject_id', 'intime', 'outtime', 'hadm_id', 'icustay_id']).reset_index(drop=True)
# 35s

C:\Users\retina\AppData\Local\Temp\ipykernel_18980\3260716528.py:23: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  icustays00 = icustays.groupby(['subject_id', 'hadm_id']).apply(


In [13]:
print(icustays.shape)
print(len(pd.unique(icustays['subject_id'])))
print(len(pd.unique(icustays['hadm_id'])))

(17081, 15)
10095
15615


In [14]:
admissions = pd.read_csv('N:\\ADMISSIONS.csv')
tmp = pd.DataFrame()
tmp['columns'] = admissions.columns
tmp['columns'] = tmp['columns'].str.lower().str.strip()
admissions.columns = tmp['columns'].to_list()

admissions = admissions[['subject_id', 'hadm_id',
                         'admittime', 'dischtime', 'deathtime',
                        'diagnosis', 'hospital_expire_flag']]

admissions['subject_id'] = admissions['subject_id'].astype('string')
admissions['subject_id'] = admissions['subject_id'].str.lower().str.strip()
admissions = admissions.dropna(subset=['subject_id'])
admissions['subject_id_int64'] = admissions['subject_id'].astype(np.int64)
admissions = admissions[admissions['subject_id_int64'].isin(
    findDf['subject_id_int64'])]

admissions['admittime'] = pd.to_datetime(
    admissions['admittime'], format='%Y-%m-%d %H:%M:%S')

admissions['dischtime'] = pd.to_datetime(
    admissions['dischtime'], format='%Y-%m-%d %H:%M:%S')

admissions['deathtime'] = pd.to_datetime(
    admissions['deathtime'], format='%Y-%m-%d %H:%M:%S')
# 1s

In [15]:
print(admissions.shape)
print(len(pd.unique(admissions['subject_id'])))
print(len(pd.unique(admissions['hadm_id'])))

(15914, 8)
10099
15914


In [16]:
patients = pd.read_csv('N:\\PATIENTS.csv')
tmp = pd.DataFrame()
tmp['columns'] = patients.columns
tmp['columns'] = tmp['columns'].str.lower().str.strip()
patients.columns = tmp['columns'].to_list()

patients = patients[['subject_id', 'gender', 'dob',
                     'dod', 'dod_hosp', 'dod_ssn', 'expire_flag']]

patients['subject_id'] = patients['subject_id'].astype('string')
patients['subject_id'] = patients['subject_id'].str.lower().str.strip()
patients = patients.dropna(subset=['subject_id'])
patients['subject_id_int64'] = patients['subject_id'].astype(np.int64)
patients = patients[patients['subject_id_int64'].isin(
    findDf['subject_id_int64'])]

patients['dob'] = pd.to_datetime(patients['dob'], format='%Y-%m-%d %H:%M:%S')
# patients['dod'] = pd.to_datetime(patients['dod'], format='%Y-%m-%d %H:%M:%S')
# patients['dod_hosp'] = pd.to_datetime(patients['dod_hosp'], format='%Y-%m-%d %H:%M:%S')
# patients['dod_ssn'] = pd.to_datetime(patients['dod_ssn'], format='%Y-%m-%d %H:%M:%S')
# 1s

In [17]:
print(patients.shape)
print(len(pd.unique(patients['subject_id'])))

(10099, 8)
10099


In [ ]:
core = admissions.merge(patients, how='outer', on=['subject_id_int64'])
core = core.merge(icustays, on=['subject_id_int64', 'hadm_id'], how='outer')


core = core.dropna(subset=['subject_id', 'hadm_id',
                           'admittime', 'dischtime',
                           'dob'])


aaa = pd.to_timedelta(core['admittime']-pd.Timestamp('now'))
bbb = pd.to_timedelta(pd.Timestamp('now') - core['dob'])
core['age'] = aaa.dt.total_seconds()+bbb.dt.total_seconds()
core['age'] = core['age'] / 60.0/60.0/24.0 / 365.24219
# https://blog.csdn.net/fyfugoyfa/article/details/125572242
# https://github.com/YerevaNN/mimic3-benchmarks/issues/84


core = core.sort_values(
    by=['subject_id', 'admittime', 'intime', 'hadm_id']).reset_index(drop=True)
core["kInd"] = range(len(core))


core["death"] = 0
zDeath = ['deathtime', 'dod', 'dod_hosp', 'dod_ssn',
          'expire_flag']
for zz in zDeath:
    ttt2 = core.dropna(subset=[zz])
    core.loc[ttt2['kInd'], 'death'] = 1

ttt2 = core.dropna(subset=['hospital_expire_flag'])
ttt2 = ttt2[ttt2['hospital_expire_flag'] == 1]
core.loc[ttt2['kInd'], 'death'] = 1

core = core[['subject_id_int64', 'subject_id', 'hadm_id', 'gender', 'age', 'death',
             'admittime', 'dischtime', 'deathtime',
            'icustay_id', 'intime', 'outtime', 'los',
             'totalCountIcu', 'numberCountIcu']]

core = core.sort_values(
    by=['subject_id', 'admittime', 'intime', 'outtime', 'dischtime', 'hadm_id']).reset_index(drop=True)

# 1s

In [19]:
print(core.shape)
print(len(pd.unique(core['subject_id'])))
print(len(pd.unique(core['hadm_id'])))

(17081, 15)
10095
15615


In [20]:
def xcGroup(inDf):
    out = pd.DataFrame()

    out.loc[0, ['subject_id', 'hadm_id']
            ] = inDf.iloc[0][['subject_id', 'hadm_id']]

    for ii in range(np.int16(inDf['seq_num'].max()+1)):
        tmp = inDf[inDf['seq_num'] == ii]
        if len(tmp) > 0:
            out.loc[0, 'icd9_code' +
                    "{:0>2d}".format(int(ii))] = tmp.iloc[0]['icd9_code']

    return out.reset_index(drop=True)

In [21]:
hospIcd = pd.read_csv('N:\\DIAGNOSES_ICD.csv')
tmp = pd.DataFrame()
tmp['columns'] = hospIcd.columns
tmp['columns'] = tmp['columns'].astype('string')
tmp['columns'] = tmp['columns'].str.lower().str.strip()
hospIcd.columns = tmp['columns'].to_list()

hospIcd = hospIcd[['subject_id', 'hadm_id', 'seq_num', 'icd9_code']]

hospIcd['subject_id'] = hospIcd['subject_id'].astype('string')
hospIcd['subject_id'] = hospIcd['subject_id'].str.lower().str.strip()
hospIcd['subject_id_int64'] = hospIcd['subject_id'].astype(np.int64)
hospIcd = hospIcd[hospIcd['subject_id_int64'].isin(findDf['subject_id_int64'])]


hospIcd['icd9_code'] = hospIcd['icd9_code'].astype("string")
hospIcd['icd9_code'] = hospIcd['icd9_code'].str.strip().str.lower()


hospIcd['seq_num'] = hospIcd['seq_num'].astype("string")
hospIcd['seq_num'] = hospIcd['seq_num'].str.strip().str.lower()
hospIcd = hospIcd.dropna(subset=['seq_num'])
hospIcd['seq_num'] = hospIcd['seq_num'].astype(np.float16)
hospIcd['seq_num'] = hospIcd['seq_num'].astype(np.int16)


hospIcd = hospIcd.dropna(subset=['subject_id', 'hadm_id', 'icd9_code'])


hospIcd = hospIcd.sort_values(
    by=['subject_id', 'hadm_id', 'seq_num', 'icd9_code']).reset_index(drop=True)

ppGroup = hospIcd.groupby(['subject_id', 'hadm_id']).apply(
    xcGroup).reset_index(drop=True)

ppGroup = ppGroup.sort_values(
    by=['subject_id', 'hadm_id']).reset_index(drop=True)
ppGroup['subject_id_int64'] = ppGroup['subject_id'].astype(np.int64)
# 2m44s

C:\Users\retina\AppData\Local\Temp\ipykernel_18980\71966349.py:33: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  ppGroup = hospIcd.groupby(['subject_id', 'hadm_id']).apply(


In [22]:
print(hospIcd.shape)
print(len(pd.unique(hospIcd['subject_id'])))
print(len(pd.unique(hospIcd['hadm_id'])))
print(hospIcd['seq_num'].max())

(214538, 5)
10099
15911
39


In [23]:
print(ppGroup.shape)
print(len(pd.unique(ppGroup['subject_id'])))
print(len(pd.unique(ppGroup['hadm_id'])))

(15911, 42)
10099
15911


In [24]:
core = core.merge(ppGroup, on=['subject_id_int64', 'hadm_id'], how='outer')
core['subject_id'] = core['subject_id_x']
core = core[['subject_id_int64', 'subject_id', 'hadm_id', 'gender', 'age', 'death',
             'admittime', 'dischtime', 'deathtime', 'icustay_id', 'intime',
            'outtime', 'los', 'totalCountIcu', 'numberCountIcu',
             'icd9_code01', 'icd9_code02', 'icd9_code03', 'icd9_code04',
             'icd9_code05', 'icd9_code06', 'icd9_code07', 'icd9_code08',
             'icd9_code09', 'icd9_code10', 'icd9_code11', 'icd9_code12',
             'icd9_code13', 'icd9_code14', 'icd9_code15', 'icd9_code16',
             'icd9_code17', 'icd9_code18', 'icd9_code19', 'icd9_code20',
             'icd9_code21', 'icd9_code22', 'icd9_code23', 'icd9_code24',
             'icd9_code25', 'icd9_code26', 'icd9_code27', 'icd9_code28',
             'icd9_code29', 'icd9_code30', 'icd9_code31', 'icd9_code32',
             'icd9_code33', 'icd9_code34', 'icd9_code35', 'icd9_code36',
             'icd9_code37', 'icd9_code38', 'icd9_code39']]

In [25]:
print(core.shape)
print(len(pd.unique(core['subject_id'])))
print(len(pd.unique(core['hadm_id'])))

(17378, 54)
10096
15912


In [26]:
core[:20]

,subject_id_int64,subject_id,hadm_id,gender,age,death,admittime,dischtime,deathtime,icustay_id,...,icd9_code30,icd9_code31,icd9_code32,icd9_code33,icd9_code34,icd9_code35,icd9_code36,icd9_code37,icd9_code38,icd9_code39
0,20,20,157681,F,75.876794,0.0,2183-04-28 09:45:00,2183-05-03 14:45:00,NaT,264490.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,30,30,104557,M,300.002569,0.0,2172-10-14 14:17:00,2172-10-19 14:37:00,NaT,225176.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,33,33,176176,M,82.394472,0.0,2116-12-23 22:30:00,2116-12-27 12:05:00,NaT,296681.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,52,52,190797,M,38.122900,0.0,2191-01-10 02:12:00,2191-01-19 16:10:00,NaT,261857.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,79,79,181542,M,52.293964,0.0,2175-09-25 23:05:00,2175-09-29 14:10:00,NaT,290076.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,85,85,112077,M,76.849786,0.0,2167-07-25 18:49:00,2167-07-30 15:24:00,NaT,291697.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,85,85,116630,M,71.452825,0.0,2162-03-02 14:04:00,2162-03-10 13:15:00,NaT,209562.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,107,107,174162,M,70.114620,0.0,2122-05-14 19:37:00,2122-05-18 15:11:00,NaT,264253.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,107,107,182383,M,69.662840,0.0,2121-11-30 19:24:00,2121-12-05 14:18:00,NaT,252542.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9,107,107,191941,M,62.886319,0.0,2115-02-20 17:41:00,2115-02-21 16:30:00,NaT,244517.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [29]:
import warnings
warnings.filterwarnings('ignore')

In [30]:
addpd = pd.DataFrame()

for ii in tqdm(range(len(findDf))):

    tmpTab = core[core['subject_id_int64'] ==
                  findDf.iloc[ii]['subject_id_int64']]

    if len(tmpTab) > 0:
        time02 = pd.DataFrame()
        time04 = pd.DataFrame()
        try:
            time01 = tmpTab[tmpTab['intime'] <= findDf.iloc[ii]['ecgDatetime']]
            time02 = time01[time01['outtime'] > findDf.iloc[ii]['ecgDatetime']]
        except:
            1+1

        try:
            time03 = tmpTab[tmpTab['admittime']
                            <= findDf.iloc[ii]['ecgDatetime']]
            time04 = time03[time03['dischtime']
                            > findDf.iloc[ii]['ecgDatetime']]
        except:
            2+2

        if len(time02) > 0:
            addpd.loc[ii, 'xIndex'] = ii
            addpd.loc[ii, time02.columns] = time02.iloc[len(time02)-1]
            addpd.loc[ii, 'testDuring'] = 'withinICU'

        elif len(time04) > 0:
            addpd.loc[ii, 'xIndex'] = ii
            time04 = time04[['subject_id_int64', 'subject_id', 'hadm_id', 'gender', 'age', 'death',
                             'admittime', 'dischtime', 'deathtime',
                            'icd9_code01',
                             'icd9_code02', 'icd9_code03', 'icd9_code04', 'icd9_code05',
                             'icd9_code06', 'icd9_code07', 'icd9_code08', 'icd9_code09',
                             'icd9_code10', 'icd9_code11', 'icd9_code12', 'icd9_code13',
                             'icd9_code14', 'icd9_code15', 'icd9_code16', 'icd9_code17',
                             'icd9_code18', 'icd9_code19', 'icd9_code20', 'icd9_code21',
                             'icd9_code22', 'icd9_code23', 'icd9_code24', 'icd9_code25',
                             'icd9_code26', 'icd9_code27', 'icd9_code28', 'icd9_code29',
                             'icd9_code30', 'icd9_code31', 'icd9_code32', 'icd9_code33',
                             'icd9_code34', 'icd9_code35', 'icd9_code36', 'icd9_code37',
                             'icd9_code38', 'icd9_code39']]
            addpd.loc[ii, time04.columns] = time04.iloc[len(time04)-1]
            addpd.loc[ii, 'testDuring'] = 'withinHospital'

        else:
            addpd.loc[ii, 'testDuring'] = 'before/afterHospital'
            '''print('OutOfICU/Hospital')
            print(findDf.iloc[ii])
            print()'''

    else:
        addpd.loc[ii, 'testDuring'] = 'miss'
        '''print('error subject_id not found')
        print(findDf.iloc[ii])
        print()'''
# 5m21s

100%|██████████| 22317/22317 [05:02<00:00, 73.89it/s] 


In [31]:
print(addpd.shape)
print(len(pd.unique(addpd['subject_id'])))
print(len(pd.unique(addpd['hadm_id'])))

(22317, 56)
9114
10051


In [32]:
joinTab = findDf.merge(addpd,
                       how='left', on=['xIndex', 'subject_id_int64'])
joinTab['subject_id'] = joinTab['subject_id_x']
joinTab = joinTab[[
    'subject_id', 'hadm_id', 'gender', 'age', 'death', 'admittime', 'dischtime', 'deathtime',
    'path', 'loadPath', 'ecgDate', 'ecgDatetime', 'sig_name',
    'icustay_id', 'intime', 'outtime', 'los', 'numberCountIcu', 'totalCountIcu', 'testDuring',
    'icd9_code01', 'icd9_code02', 'icd9_code03', 'icd9_code04', 'icd9_code05', 'icd9_code06', 'icd9_code07', 'icd9_code08', 'icd9_code09', 'icd9_code10',
    'icd9_code11', 'icd9_code12', 'icd9_code13', 'icd9_code14', 'icd9_code15', 'icd9_code16', 'icd9_code17', 'icd9_code18', 'icd9_code19', 'icd9_code20',
    'icd9_code21', 'icd9_code22', 'icd9_code23', 'icd9_code24', 'icd9_code25', 'icd9_code26', 'icd9_code27', 'icd9_code28', 'icd9_code29', 'icd9_code30',
    'icd9_code31', 'icd9_code32', 'icd9_code33', 'icd9_code34', 'icd9_code35', 'icd9_code36', 'icd9_code37', 'icd9_code38', 'icd9_code39']]

joinTab = joinTab.sort_values(
    by=['subject_id', 'admittime', 'ecgDatetime', 'intime', 'outtime', 'dischtime', 'hadm_id'])

In [33]:
print(joinTab.shape)
print(len(pd.unique(joinTab['subject_id'])))
print(len(pd.unique(joinTab['hadm_id'])))
print(len(pd.unique(joinTab['icustay_id'])))

(22317, 59)
10099
10051
9632


In [34]:
joinTab['testDuring'].value_counts()

testDuring
withinICU         15789
withinHospital     2163
Name: count, dtype: int64

In [35]:
tt = joinTab[joinTab['testDuring'] == 'withinICU']
print(tt.shape)
print(len(pd.unique(tt['subject_id'])))
print(len(pd.unique(tt['hadm_id'])))
print(len(pd.unique(tt['icustay_id'])))

(15789, 59)
8486
9306
9631


In [36]:
saveTimeStr = time.strftime("%Y%m%d_%H%M%S", time.localtime())
joinTab.to_csv(
    saveTimeStr+'mimic-iii-wave-match-BasicTable-icd.csv', index=False)

In [ ]:
'''The MIMIC-III Waveform Database Matched Subset contains 22,317 waveform records, 
and 22,247 numerics records, 
for 10,282 distinct ICU patients. 
https://physionet.org/content/mimic3wdb-matched/1.0/
'''